# Data Blog: Exploratory Data Analysis Workshop in Python

In this workshop, we will be analyzing Melbourne Housing data to see if real estate is really *booming* (as described by the owner of the dataset). This dataset is from Kaggle, found [here](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot).

## Step 0: Import all the Needed Packages

Before starting any project, always make sure you have all the proper packages to manipulate the dataset and create visualizations.

In [ ]:
import pandas as pd #used for working with dataframes—our dataset
import numpy as np #used to manipulate arrays
import matplotlib.pyplot as plt #for making visuals
import seaborn as sns #for making visuals too

(Tip: use the "Run" button to run a cell, or the shortcut command-R, if on a Mac, with the cell highlighted to run it)

## Step 1: Loading the Data

Once you found the necessary data, the next step is to load the data on jupyter. There are various ways you can load the data depending on where it came from:
* Download the data manually and load it from its location in your files
* Use the `urlretrieve` function from the `urllib.request` to download CSV files from a raw URL

In most cases, it's easiest to download the data in the same folder as your jupyter notebook (as we had already done) and just load it with the following command:

In [ ]:
melbourne_df = pd.read_csv('melb_data.csv')

We can then take a look at the first few rows of our dataframe by issuing the following command:

In [ ]:
melbourne_df.head()

Now our dataframe is saved in the variable `melborne_df` and we can use it analyze the data. Let's look at the columns our dataset.

In [ ]:
melbourne_df.columns

## Step 2: Data Cleaning

After looking over the data, let's start cleaning it.

We'll begin by only taking a subset of the data that we actually want to work with.

In [ ]:
# list of column names we want
cols = ['Rooms', 'Type', 'Price', 'Date', 'Bathroom', 'Car', 'Regionname',
       'Landsize', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude', 'Propertycount']

In [ ]:
melbourne_df = melbourne_df[cols].copy()

In [ ]:
melbourne_df.shape

Now let's see the information on our data.

In [ ]:
melbourne_df.info()

A lot of the data in integer/float data, but we can see that `Date`, `Type`, and `CouncilArea` are object type. There is a date type in pandas we can use which many be useful as well as a function that is useful for categorical variables like `Type`.

We can also see that some columns have less than 13580 non-null values, meaning some infomation is missing. Let's deal with that first.

The YearBuilt has the most NA values, and we would like to work with that data. We will drop all the rows that are NA for year built (and we understand we will be losing a lot of data).

In [ ]:
melbourne_df.dropna(subset=['YearBuilt'], inplace=True) 
#note that inplace=True is need to change the ACTUAL dataset, or else a copy will be returned instead
melbourne_df

In [ ]:
melbourne_df.info()

Now there's only a few of the `Car` values missing (the number of car spaces on the lot). For this column we are going to replace the NA values with the median number of car spaces.

In [ ]:
melbourne_df['Car'].fillna(melbourne_df['Car'].median(), inplace=True)
melbourne_df

In [ ]:
melbourne_df.info()

For `CouncilArea`, let's just drop the whole column.

In [ ]:
melbourne_df.drop(columns=['CouncilArea'], inplace=True)
melbourne_df

In [ ]:
melbourne_df.info()

Let's look over the integer data and see what needs to be fixed.

In [ ]:
melbourne_df.describe()

In [ ]:
melbourne_df = melbourne_df[melbourne_df['Landsize'] > 100]
#at least 100 square feet

In [ ]:
melbourne_df = melbourne_df[melbourne_df['Bathroom'] > 0]
#at least 1 bathroom

In [ ]:
melbourne_df = melbourne_df[melbourne_df['YearBuilt'] > 1196]
#1196 does not sounds right

In [ ]:
melbourne_df.describe()

Now, let's fix the Date and YearBuilt datatypes.

In [ ]:
melbourne_df['YearBuilt'] =  pd.to_datetime(dict(year=melbourne_df['YearBuilt'], month=1, day=1))
#using 1 for month and day since DateTime requires these values

In [ ]:
melbourne_df['YearBuilt']

In [ ]:
melbourne_df['Date'] =  pd.to_datetime(melbourne_df['Date'], format='%d/%m/%Y')

In [ ]:
melbourne_df['Date']

Next, we'll deal with the Type column. Since there are different values, we will be using Pandas' `get_dummies` function to make a column for each categorical value.

In [ ]:
melbourne_df = pd.get_dummies(melbourne_df, prefix=['type'], columns=['Type'])
melbourne_df

## Step 3: Exploratory Analysis and Visualization

Now we will do some exploratory data analysis to see what information we can gather from our data before we start asking more specific questions.

Let's try answering the following questions:

1. What is the average number of rooms in this dataset and what is the median number of rooms? (Hint: use [] or . to select a column from the dataset)

In [ ]:
meanrooms = melbourne_df['Rooms'].mean()
medianrooms = ???

print(f'The average number of rooms for a property is Melbourne is {meanrooms} and the most common number of rooms is {medianrooms}')

2. What is the average price of properties built after 1970? (Hint: in the data cleaning process, we used a function to only look at data where a column value was above a certain threshold)

In [ ]:
avgprice = ???

print(f'The average price of a property built after 1970 is {avgprice}')

3. What is the average price of property by region, given that the property has more than 2 rooms? (Hint: the [groupby function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) is very useful and the answer should be a Series—one column)

In [ ]:
???

Now, let's see what sorts of visualizations we can make. First, let's plot of histogram of the prices of properties in Melbourne.

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Prices of Properties in Melbourne')
plt.xlabel('Price')
plt.ylabel('Number of properties')

plt.hist(melbourne_df['Price'], melbourne_df['Price'].count(), color='purple');

Seems like outliers have made the graph pretty useless. Query the dataframe so that we only see a histogram of properties that fall within the IQR (25%-75%). (Hint: Use the [quantile function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html))

In [ ]:
IQRprices_df = ???

plt.figure(figsize=(12, 6))
plt.title('Prices of Properties in Melbourne')
plt.xlabel('Price')
plt.ylabel('Number of properties')

plt.hist(IQRprices_df['Price'], IQRprices_df['Price'].count(), color='purple');

That's a little better. In the future, we can use [subplots](https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplots_demo.html) to plot the range of prices for each type of property, or by each region to really understand the price distribution.

Let's make a visual of the number of houses built over the years.

In [ ]:
#make a new dataframe with date information
timebuilt_df = melbourne_df.groupby('YearBuilt').count().reset_index()[['YearBuilt','Rooms']]
timebuilt_df['Built'] = timebuilt_df['Rooms']
timebuilt_df['Year'] = pd.DatetimeIndex(timebuilt_df['YearBuilt']).year
timesold_df = melbourne_df.groupby('Date').count().reset_index()[['Date','Rooms']]
timesold_df['Year'] = pd.DatetimeIndex(timesold_df['Date']).year
timesold_df['Sold'] = timesold_df['Rooms']
time_df= timebuilt_df.merge(timesold_df, on='Year', how='left')
time_df.drop(columns=['Rooms_x', 'Rooms_y', 'YearBuilt', 'Date'], inplace=True)
time_df=time_df.groupby('Year').sum()[['Built', 'Sold']].reset_index()
time_df

In [ ]:
sns.set_style("darkgrid")
sns.lineplot(data=time_df, x='Year', y='Built');
sns.lineplot(data=time_df, x='Year', y='Sold');
plt.legend(labels=["Properties Built", "Properties Sold"])
plt.ylabel('Count')
plt.title('Properties Sold vs Built');

Finally, using another cool function from `Plotly`, let's map the properties in Melbourne.

In [ ]:
import plotly.express as px


fig = px.scatter_mapbox(melbourne_df, lat="Lattitude", lon="Longtitude", hover_data=["Price", "Landsize"],
                        color="Price", size="Landsize", zoom=10)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## More Resources

We only scratched the surface of what you can do with EDA in Python. We recommend looking at functions like the [apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) function and the [merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) function for working with complexer queries and multiple datasets. And remember, StackOverflow and Google are your friends!